In [ ]:
import mne 
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,Conv2D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
import matplotlib.pyplot as plt
import seaborn as sn 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler



allDataJoy = pd.read_csv("ExperimentoMarioPostFeaturesSim/joy_procesedSimulador.csv")  
allDataAngry = pd.read_csv('ExperimentoMarioPostFeaturesSim/angry_procesedSimulador.csv')  
allDataNeutral = pd.read_csv('ExperimentoMarioPostFeaturesSim/neutral_procesedSimulador.csv')  
allDataNoStim = pd.read_csv('ExperimentoMarioPostFeaturesSim/nada_procesedSimulador.csv')  

In [ ]:
allDataNoStim = allDataNoStim.assign(Emotion = 0)
allDataAngry = allDataAngry.assign(Emotion = 1)
allDataNeutral = allDataNeutral.assign(Emotion = 2)
allDataJoy = allDataJoy.assign(Emotion = 3)

In [ ]:
# We divide each of the datasets into train and test to have balanced training and validation sets

joyCopia = allDataJoy.copy()
angerCopia = allDataAngry.copy()
neutralCopia = allDataNeutral.copy()
nadaCopia = allDataNoStim.copy()


yJoy = joyCopia[['Emotion']]

xJoy = joyCopia.drop(columns = ["Emotion"])

X_trainJoy, X_testJoy, y_trainJoy, y_testJoy = train_test_split(xJoy,yJoy, test_size=0.20, random_state=42, shuffle=False)

In [ ]:
yAngry = angerCopia[['Emotion']]

xAngry = angerCopia.drop(columns = ["Emotion"])

X_trainAngry, X_testAngry, y_trainAngry, y_testAngry = train_test_split(xAngry,yAngry, test_size=0.20, random_state=42, shuffle=False)

In [ ]:
yNeutral = neutralCopia[['Emotion']]

xNeutral = neutralCopia.drop(columns = ["Emotion"])

X_trainNeutral, X_testNeutral, y_trainNeutral, y_testNeutral = train_test_split(xNeutral,yNeutral, test_size=0.20, random_state=42, shuffle=False)

In [ ]:
yNada = nadaCopia[['Emotion']]

xNada = nadaCopia.drop(columns = ["Emotion"])

X_trainNada, X_testNada, y_trainNada, y_testNada = train_test_split(xNada,yNada, test_size=0.20, random_state=42, shuffle=False)

In [ ]:
# Data with 4 emotions

x_train = pd.concat([X_trainJoy.copy(), X_trainAngry.copy(), X_trainNeutral.copy(), X_trainNada.copy()], axis=0)
y_train = pd.concat([y_trainJoy.copy(), y_trainAngry.copy(), y_trainNeutral.copy(), y_trainNada.copy()], axis=0)
x_test = pd.concat([X_testJoy,X_testAngry,X_testNeutral, X_testNada], axis=0)
y_test = pd.concat([y_testJoy,y_testAngry,y_testNeutral, y_testNada], axis=0)

# Training with LSTM model

In [ ]:

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))

model.add(Flatten())
#model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer="adam", loss= "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(X_train, y_train,  epochs=3, validation_split=0.15, callbacks=[es])

In [ ]:
X_test, y_test = np.array(x_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
X_test, y_test = np.array(x_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
accuracy_score(y_test, y_train_pred)

In [ ]:
print(classification_report(y_test, y_train_pred))

# CNN-based model

In [ ]:
model = Sequential()#add model layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(Conv1D(32, kernel_size=3, activation='relu'))

model.add(Flatten())

model.add(Dense(4, activation='softmax'))
model.compile(optimizer="adam", loss= "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train,  epochs=3, validation_split=0.15, callbacks=[es])

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
print(classification_report(y_test, y_train_pred))

# Model based on CNN + LSTM

In [ ]:
model = Sequential()#add model layers
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(288,1)))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.compile(optimizer="adam", loss= "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
print(classification_report(y_test, y_train_pred))

# Next, the ternary classification

In [ ]:
# Data with three emotions

x_train = pd.concat([X_trainAngry.copy(), X_trainNeutral.copy(), X_trainNada.copy()], axis=0)
y_train = pd.concat([ y_trainAngry.copy(), y_trainNeutral.copy(), y_trainNada.copy()], axis=0)
x_test = pd.concat([X_testAngry,X_testNeutral, X_testNada], axis=0)
y_test = pd.concat([y_testAngry,y_testNeutral, y_testNada], axis=0)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))

model.add(Flatten())
#model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer="adam", loss= "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(X_train, y_train,  epochs=3, validation_split=0.15, callbacks=[es])

In [ ]:
X_test, y_test = np.array(x_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
X_test, y_test = np.array(x_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
accuracy_score(y_test, y_train_pred)

In [ ]:
print(classification_report(y_test, y_train_pred))

# CNN-based model

In [ ]:
model = Sequential()#add model layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(Conv1D(32, kernel_size=3, activation='relu'))

model.add(Flatten())

model.add(Dense(3, activation='softmax'))
model.compile(optimizer="adam", loss= "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train,  epochs=3, validation_split=0.15, callbacks=[es])

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
print(classification_report(y_test, y_train_pred))

# Model based on CNN + LSTM

In [ ]:
model = Sequential()#add model layers
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(288,1)))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(optimizer="adam", loss= "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

# Binary classification

In [ ]:
# Data with two emotions

x_train = pd.concat([X_trainAngry.copy(),  X_trainNada.copy()], axis=0)
y_train = pd.concat([ y_trainAngry.copy(), y_trainNada.copy()], axis=0)
x_test = pd.concat([X_testAngry, X_testNada], axis=0)
y_test = pd.concat([y_testAngry, y_testNada], axis=0)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))

model.add(Flatten())
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss= "binary_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(X_train, y_train,  epochs=3, validation_split=0.15, callbacks=[es])

In [ ]:
X_test, y_test = np.array(x_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
X_test, y_test = np.array(x_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
accuracy_score(y_test, y_train_pred)

In [ ]:
print(classification_report(y_test, y_train_pred))

# CNN-based model

In [ ]:
model = Sequential()#add model layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(Conv1D(32, kernel_size=3, activation='relu'))

model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss= "binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train,  epochs=3, validation_split=0.15, callbacks=[es])

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
print(classification_report(y_test, y_train_pred))

# Model based on CNN + LSTM

In [ ]:
model = Sequential()#add model layers
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(288,1)))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss= "binary_crossentropy", metrics=["accuracy"])

In [ ]:
y_train_pred = model.predict_classes(X_test)

In [ ]:
matrix = confusion_matrix(y_test, y_train_pred)

sn.heatmap (matrix, annot = True, cmap='Blues', fmt="d")

In [ ]:
print(classification_report(y_test, y_train_pred))